## Project Setup

In [1]:
# all import statements needed for the project
from math import *
from math import sin, cos, sqrt, atan2, radians
import numpy as np
import bs4
import matplotlib.pyplot as plt
import pandas as pd
import requests
import sqlalchemy as db
import warnings
import datetime
from tqdm import tqdm
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'numpy'

In [ ]:
# Constants to refer to any local data
TAXI_URL = "https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page"
UBER_CSV = "uber_rides_sample.csv"

NEW_YORK_BOX_COORDS = ((40.560445, -74.242330), (40.908524, -73.717047))

DATABASE_URL = "sqlite:///project.db"
DATABASE_SCHEMA_FILE = "schema.sql"
QUERY_DIRECTORY = "queries"

In [ ]:
# Transfer Location ID to Longitude and Latitude
import geopandas as gpd
df= gpd.read_file("taxi_zones/taxi_zones.shp")
df=df.to_crs(epsg=4326)
df['Center_point']=df['geometry'].centroid
df["long"]=df.Center_point.map(lambda p:p.x)
df["lat"]=df.Center_point.map(lambda p:p.y)
df

In [ ]:
# Let object id start from 1
df.index[df['OBJECTID'] == 57].values[0]

## Part 1: Data Preprocessing

### Calculating distance
Not all data source we have with distance, so we calculate distance for those source without distance and add it to our dataframe

In [ ]:
def calculate_distance(from_coord, to_coord):
    """
    Calculate the distance between two coordinates
    """
    x0 = from_coord[0]
    y0 = from_coord[1]
    x1 = to_coord[0]
    y1 = to_coord[1]
    R = 6373.0
    lat0 = radians(y0)
    lon0 = radians(x0)
    lat1 = radians(y1)
    lon1 = radians(x1)
    dlon = lon1 - lon0
    dlat = lat1 - lat0
    a = sin(dlat / 2)**2 + cos(lat0) * cos(lat1) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c
    return distance

In [ ]:
def add_distance_column(df):
    """
    Add the distance calculated to dataframe
    """
    from_coord = ["pickup_longitude","pickup_latitude"]
    to_coord = ["dropoff_longitude","dropoff_latitude"]
    df['distance'] = np.nan
    for index, row in df.iterrows():
        df.loc[index,'distance'] = calculate_distance([row["pickup_longitude"],row["pickup_latitude"]],[row["dropoff_longitude"],row["dropoff_latitude"]])                                       
    return df

### Processing Taxi Data

Download, clean, and and sample Taxi Data

In [ ]:
import bs4
import requests
def get_taxi_html(TAXI_URL):
    """
    Get taxi data from website
    """
    response = requests.get(TAXI_URL)
    html = response.content
    return html

In [ ]:
def find_taxi_parquet_links(TAXI_URL):
    """
    Find parquet link with yellow taxi from html
    """
    soup=bs4.BeautifulSoup(get_taxi_html(TAXI_URL), 'html.parser')
    res=[]
    par=soup.find_all('a')
    for i in par:
        if 'Yellow Taxi Trip Records' in i:
            res.append(i.get('href'))
    return res

In [ ]:
import re
def find_taxi_csv_urls(TAXI_URL):
    """
    Find URLs in a string
    """
    return find_taxi_parquet_links(TAXI_URL)

In [ ]:
def get_and_clean_month_taxi_data(data_path):
    """
    Read parquet files in pandas
    """
    return pd.read_parquet(data_path, engine='pyarrow')

In [ ]:
def combine(df,obj,sub):
    """
    Merge parquets files into a dateframe
    """
    for index, row in df.iterrows():
        for name in sub:
            if df.loc[index,name] == df.loc[index,name]:
                df.loc[index,obj] = df.loc[index,name]
    return df

In [ ]:
import os.path
def get_and_clean_taxi_data(TAXI_URL,df):
    """
    Download, clean, and and sample Taxi dataframe by removing invalid data and normalizing column names
    """
    taxi_data = pd.DataFrame(columns=['tpep_pickup_datetime', 'tpep_dropoff_datetime','passenger_count', 'trip_distance', 
           'PULocationID', 'DOLocationID', 'fare_amount','tip_amount', 'tolls_amount', 
           'total_amount','pickup_longitude','pickup_latitude',  'dropoff_longitude', 'dropoff_latitude']) 
    all_csv_urls = find_taxi_csv_urls(TAXI_URL)
    for csv_url in all_csv_urls:
        file_name = re.search('https://d37ci6vzurychx.cloudfront.net/trip-data/(.*).parquet', csv_url).group(1)
        if  int(file_name[-7:-3])<2015 or (int(file_name[-7:-3])==2015 and int(file_name[-2:])<=6):
            if os.path.isfile('./'+file_name+'.parquet'):
            # maybe: first try to see if you've downloaded this exact
            # file already and saved it before trying again
                print(file_name," saved")
                dataframe = pd.read_parquet(file_name+'.parquet')
                #dataframe = dataframe.sample(n=100, random_state=1)
            else:
                print(file_name," downloading")
                dataframe = get_and_clean_month_taxi_data(csv_url)
                dataframe = dataframe.sample(n=3000, random_state=1)
                dataframe.to_parquet(file_name+'.parquet')
            if int(file_name[-7:-3])==2010:
                dataframe.rename(columns={"pickup_datetime": "tpep_pickup_datetime", "dropoff_datetime": "tpep_dropoff_datetime"},inplace=True)
            elif int(file_name[-7:-3])==2009:
                dataframe.rename(columns={"Trip_Pickup_DateTime": "tpep_pickup_datetime", "Trip_Dropoff_DateTime": "tpep_dropoff_datetime",
                                         "Passenger_Count":"passenger_count","Trip_Distance": "trip_distance", 
                                         "Start_Lon": "pickup_longitude","Start_Lat":"pickup_latitude",
                                         "End_Lon": "dropoff_longitude","End_Lat":"dropoff_latitude",
                                         "Fare_Amt": "fare_amount","Tip_Amt":"tip_amount",
                                         "Tolls_Amt": "tolls_amount","Total_Amt":"total_amount"},inplace=True)
            else:
                for index,row in dataframe.iterrows():
                    SID = int(dataframe.loc[index,'PULocationID'])
                    EID = int(dataframe.loc[index,'DOLocationID'])
                    if SID<=263 and EID<=263 and SID>=1 and EID>=1:
                        S_index = df.index[df['OBJECTID'] == SID].values[0]
                        E_index = df.index[df['OBJECTID'] == EID].values[0]
                        dataframe.loc[index,'pickup_longitude'] = df.loc[S_index,'long']
                        dataframe.loc[index,'pickup_latitude'] = df.loc[S_index,'lat']
                        dataframe.loc[index,'dropoff_longitude'] = df.loc[E_index,'long']
                        dataframe.loc[index,'dropoff_latitude'] = df.loc[E_index,'lat']
                    else:
                        dataframe.drop(index=index,inplace=True)
            taxi_data = taxi_data.append(dataframe)
            taxi_data = taxi_data[['tpep_pickup_datetime', 'tpep_dropoff_datetime','passenger_count', 'trip_distance', 
           'PULocationID', 'DOLocationID', 'fare_amount','tip_amount', 'tolls_amount', 
           'total_amount','pickup_longitude','pickup_latitude',  'dropoff_longitude', 'dropoff_latitude']]         
    taxi_data.fillna(0,inplace=True)
    return taxi_data

### Processing Uber Data

Download and clean Uber Data

In [ ]:
def load_and_clean_uber_data(csv_file):
    """
    Download and clean Uber dataframe by removing invalid data and normalizing column names
    """
    data = pd.read_csv(csv_file)
    # remove trips outside of the constraint
    data = data[(data['pickup_longitude']>=-74.242330) & (data['pickup_longitude']<=-73.717047) & 
                (data['pickup_latitude']>=40.560445) & (data['pickup_latitude']<=40.908524) &
                (data['dropoff_longitude']>=-74.242330) & (data['dropoff_longitude']<=-73.717047) & 
                (data['dropoff_latitude']>=40.560445) & (data['dropoff_latitude']<=40.908524)]
    # select useful columns
    data = data[['fare_amount','pickup_datetime','pickup_longitude','pickup_latitude','dropoff_longitude','dropoff_latitude','passenger_count']]
    # change to the correct datatype
    data['pickup_datetime'] = pd.to_datetime(data['pickup_datetime'], format='%Y-%m-%d %H:%M:%S %Z')
    data['pickup_datetime'] = data['pickup_datetime'].dt.tz_localize(None)
    
    data.fillna(0,inplace=True)
    return data

In [ ]:
def get_uber_data(UBER_DATA):
    """
    Save the clean dataframe
    """
    if os.path.isfile('./'+'uber_dataframe.xlsx'):
        uber_dataframe = pd.read_excel('uber_dataframe.xlsx')
    else:
        uber_dataframe = load_and_clean_uber_data(UBER_DATA)
        add_distance_column(uber_dataframe)
        uber_dataframe.to_excel('uber_dataframe.xlsx')
    return uber_dataframe

### Processing Weather Data

Download and clean Weather Data

In [ ]:
def clean_month_weather_data_hourly(csv_file):
    """
    Download and clean hourly weather data
    """
    all_df = pd.read_csv(csv_file+'.csv')
    # select useful columns
    all_df = all_df[['STATION','DATE','LATITUDE','LONGITUDE','NAME','HourlyWindSpeed','HourlyPrecipitation']]
    # change to the correct datatype
    all_df['DATE'] = pd.to_datetime(all_df['DATE'], format='%Y-%m-%d %H:%M:%S')
    # deal with missing data
    all_df.fillna(0,inplace=True)
    mask = all_df['HourlyPrecipitation'] == "T"
    all_df.loc[mask, 'HourlyPrecipitation'] = 0
    all_df['HourlyPrecipitation'] = all_df['HourlyPrecipitation'].map(lambda x: str(x).rstrip('s'))
    all_df['HourlyPrecipitation'] = all_df['HourlyPrecipitation'].astype("float64")
    # add new columns for classification
    all_df['day'] = all_df['DATE'].dt.date
    all_df['hour'] = all_df['DATE'].dt.hour
    # classification grouped by hour and date
    all_df = all_df.groupby([all_df['day'],all_df['hour']]).agg({'HourlyWindSpeed':'mean', 'HourlyPrecipitation':'sum'})
    return all_df

In [ ]:
def clean_month_weather_data_daily(csv_file):
    """
    Download and clean daily weather data
    """
    all_df = pd.read_csv(csv_file+'.csv')
    # select useful columns
    all_df = all_df[['STATION','DATE','LATITUDE','LONGITUDE','NAME','HourlyWindSpeed','HourlyPrecipitation']]
    # change to the correct datatype
    all_df['DATE'] = pd.to_datetime(all_df['DATE'], format='%Y-%m-%d %H:%M:%S')
    # deal with missing data
    all_df.fillna(0,inplace=True)
    mask = all_df['HourlyPrecipitation'] == "T"
    all_df.loc[mask, 'HourlyPrecipitation'] = 0
    all_df['HourlyPrecipitation'] = all_df['HourlyPrecipitation'].map(lambda x: str(x).rstrip('s'))
    all_df['HourlyPrecipitation'] = all_df['HourlyPrecipitation'].astype("float64")
    # add new columns for classification
    all_df['day'] = all_df['DATE'].dt.date
    # classification grouped by hour and date
    all_df = all_df.groupby([all_df['day']]).agg({'HourlyWindSpeed':'mean', 'HourlyPrecipitation':'sum'})
    return all_df

In [ ]:
def load_and_clean_weather_data():
    """
    Save clean weather data to dataframe 
    """
    hourly_dataframes = []
    daily_dataframes = []
    # add some way to find all weather CSV files
    # or just add the name/paths manually
    weather_csv_files = ['2009_weather','2010_weather','2011_weather','2012_weather','2013_weather','2014_weather','2015_weather']  
    for csv_file in weather_csv_files:
        hourly_dataframe = clean_month_weather_data_hourly(csv_file)
        daily_dataframe = clean_month_weather_data_daily(csv_file)
        hourly_dataframes.append(hourly_dataframe)
        daily_dataframes.append(daily_dataframe)    
    # create two dataframes with hourly & daily data from every month
    hourly_data = pd.concat(hourly_dataframes)
    daily_data = pd.concat(daily_dataframes)
    return hourly_data, daily_data

### Process All Data

Execute all the required functions to download and save our clean data

In [ ]:
taxi_data = get_and_clean_taxi_data(TAXI_URL,df)
uber_data = get_uber_data(UBER_CSV)
hourly_weather_data, daily_weather_data = load_and_clean_weather_data()

In [ ]:
taxi_data

In [ ]:
taxi_data.to_excel('test.xlsx')

In [ ]:
uber_data

In [ ]:
hourly_weather_data.reset_index(inplace=True)

In [ ]:
hourly_weather_data

In [ ]:
daily_weather_data.reset_index(inplace=True)

In [ ]:
daily_weather_data

## Part 2: Storing Cleaned Data

Create tables to store cleaned data

In [ ]:
engine = db.create_engine(DATABASE_URL)

In [ ]:
# Create your 4 tables
HOURLY_WEATHER_SCHEMA = """
CREATE TABLE IF NOT EXISTS hourly_weather (
   hourId INTEGER PRIMARY KEY,
   day DATE,
   hour INTEGER,
   HourlyWindSpeed FLOAT,
   HourlyPrecipitation FLOAT
);
"""

DAILY_WEATHER_SCHEMA = """
CREATE TABLE IF NOT EXISTS daily_weather (
   dayId INTEGER PRIMARY KEY,
   day DATE,
   HourlyWindSpeed FLOAT,
   HourlyPrecipitation FLOAT
);
"""

TAXI_TRIPS_SCHEMA = """
CREATE TABLE IF NOT EXISTS taxi_trips (
   taxiId INTEGER PRIMARY KEY,
   tpep_pickup_datetime DATETIME,
   tpep_dropoff_datetime DATETIME,
   passenger_count INTEGER,
   trip_distance FLOAT,
   PULocationID INTEGER,
   DOLocationID INTEGER,
   fare_amount FLOAT,
   tip_amount FLOAT,
   tolls_amount FLOAT,
   total_amount FLOAT,
   pickup_longitude FLOAT,
   pickup_latitude FLOAT,
   dropoff_longitude FLOAT,
   dropoff_latitude FLOAT
);
"""

UBER_TRIPS_SCHEMA = """
CREATE TABLE IF NOT EXISTS uber_trips (
   uberId INTEGER PRIMARY KEY,
   fare_amount FLOAT,
   pickup_datetime DATETIME,
   pickup_longitude FLOAT,
   pickup_latitude FLOAT,
   dropoff_longitude FLOAT,
   dropoff_latitude FLOAT,
   passenger_count INTEGER,
   distance FLOAT
);
"""

In [ ]:
# Create that required schema.sql file
with open(DATABASE_SCHEMA_FILE, "w") as f:
    f.write(HOURLY_WEATHER_SCHEMA)
    f.write(DAILY_WEATHER_SCHEMA)
    f.write(TAXI_TRIPS_SCHEMA)
    f.write(UBER_TRIPS_SCHEMA)

In [ ]:
# Create the tables with the schema files
with engine.connect() as connection:
    connection.execute(HOURLY_WEATHER_SCHEMA)
    connection.execute(DAILY_WEATHER_SCHEMA)
    connection.execute(TAXI_TRIPS_SCHEMA)
    connection.execute(UBER_TRIPS_SCHEMA)

### Add Data to Database

Add data to the table we just created

In [ ]:
def write_dataframes_to_table(table_to_df_dict):
    """
    Add dataframes to the table we just created
    """
    with engine.connect() as connection:
        for table_name in [ "taxi_trips", "uber_trips", "hourly_weather", "daily_weather"]:
            current_df = table_to_df_dict[table_name]
            for i in tqdm(range(len(current_df))):
                data_peice = [current_df.iloc[i][key] for key in current_df.columns]
                #print(tuple(data_peice))
                for j in range(len(data_peice)):
                    if isinstance(data_peice[j], datetime.date):
                        #print("triggered")
                        sql_date = str(data_peice[j])
                        data_peice[j] = sql_date
                        pass
                data_peice = [i+1] + data_peice
                sql_command = 'insert into {} values {};'.format(table_name, tuple(data_peice))
                # print(sql_command)
                connection.execute(sql_command)

In [ ]:
# process entire session
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session = Session()
cursor = session.execute("select * from uber_trips")
result = cursor.fetchall()
session.close()
print(result)

In [ ]:
del uber_data['Unnamed: 0']

In [ ]:
map_table_name_to_dataframe = {
    "taxi_trips": taxi_data,
    "uber_trips": uber_data,
    "hourly_weather": hourly_weather_data,
    "daily_weather": daily_weather_data,
}

In [ ]:
write_dataframes_to_table(map_table_name_to_dataframe)

## Part 3: Understanding the Data

In [ ]:
uber_data_table_name = uber_data 
taxi_data_table_name = taxi_data
hour_weather_table_name = hourly_weather_data
daily_weather_table_name = daily_weather_data

In [ ]:
def write_query_to_file(query, outfile):
    with open(outfile,'w') as f:
        f.write(query)

### Query 1
The most popular hour of the day to take a yellow taxi from 01-2009 to 06-2015

In [ ]:
QUERY_1 = """
SELECT STRFTIME('%H', tpep_pickup_datetime) as hour, COUNT(*) as ORDERPERHOUR
FROM taxi_trips
GROUP BY STRFTIME('%H', tpep_pickup_datetime)
"""

In [ ]:
res = engine.execute(QUERY_1).fetchall()
# write_query_to_file(QUERY_N, "query_1.sql")
res

### Query 2
The most popular day of the week to take an uber

In [ ]:
QUERY_2 = """
SELECT STRFTIME('%w', pickup_datetime) as weekday,
       COUNT(*) as ORDERWEEKDAY
FROM uber_trips
GROUP BY STRFTIME('%w', pickup_datetime)
"""

In [ ]:
res = engine.execute(QUERY_2).fetchall()
# write_query_to_file(QUERY_N, "query_2.sql")
res

### Query 3
The 95% percentile of distance traveled for all hired trips during July 2013

In [ ]:
QUERY_3 = """
SELECT hired_data.distance AS '95% distance'
FROM 
(
SELECT tpep_pickup_datetime as pickup_datetime, trip_distance as distance 
FROM taxi_trips WHERE STRFTIME('%Y', tpep_pickup_datetime) = '2013' AND STRFTIME('%m', tpep_pickup_datetime) = '07'
UNION ALL
SELECT pickup_datetime,distance 
FROM uber_trips WHERE STRFTIME('%Y', pickup_datetime) = '2013' AND STRFTIME('%m', pickup_datetime) = '07'
) as hired_data  
ORDER BY hired_data.distance ASC
LIMIT 1
OFFSET (SELECT
         COUNT(*)
        FROM (
            SELECT tpep_pickup_datetime as pickup_datetime, trip_distance as distance 
            FROM taxi_trips WHERE STRFTIME('%Y', tpep_pickup_datetime) = '2013' AND STRFTIME('%m', tpep_pickup_datetime) = '07'
            UNION ALL
            SELECT pickup_datetime, distance 
            FROM uber_trips WHERE STRFTIME('%Y', pickup_datetime) = '2013' AND STRFTIME('%m', pickup_datetime) = '07'
             ) as hired_data  
        ) * 95 / 100 - 1;
"""

In [ ]:
res = engine.execute(QUERY_3).fetchall()
# write_query_to_file(QUERY_N, "query_3.sql")
res

### Query 4
The top 10 days with the highest number of hired rides for 2009
The average distance for each day

In [ ]:
QUERY_4 = """
SELECT hired_data.pickup_datetime, COUNT(*), AVG(hired_data.distance) 
FROM 
(
    SELECT tpep_pickup_datetime as pickup_datetime, trip_distance as distance 
    FROM taxi_trips WHERE STRFTIME('%Y', tpep_pickup_datetime) = '2009'
    UNION ALL
    SELECT pickup_datetime, distance 
    FROM uber_trips WHERE STRFTIME('%Y', pickup_datetime) = '2009'
)as hired_data GROUP BY STRFTIME('%j', hired_data.pickup_datetime) 
HAVING COUNT(*) IN (  SELECT COUNT(*)
    FROM 
    (
        SELECT tpep_pickup_datetime as pickup_datetime, trip_distance as distance 
        FROM taxi_trips WHERE STRFTIME('%Y', tpep_pickup_datetime) = '2009'
        UNION ALL
        SELECT pickup_datetime, distance 
        FROM uber_trips WHERE STRFTIME('%Y', pickup_datetime) = '2009'
    )as hired_data 
    GROUP BY STRFTIME('%j', hired_data.pickup_datetime)
    ORDER BY count(*) DESC
    LIMIT 10);
"""

In [ ]:
res=engine.execute(QUERY_4).fetchall()
# write_query_to_file(QUERY_N, "query_4.sql")
res

### Query 5
The windiest 10 days in 2014
Number of hired trips were made on those days

In [ ]:
QQUERY_5 = """

SELECT hired_data.pickup_datetime, COUNT(*) 

FROM(
        SELECT tpep_pickup_datetime as pickup_datetime, trip_distance as distance 
        FROM taxi_trips WHERE STRFTIME('%Y', tpep_pickup_datetime) = '2014'
        UNION ALL
        SELECT pickup_datetime, distance 
        FROM uber_trips WHERE STRFTIME('%Y', pickup_datetime) = '2014'
)
as hired_data 
INNER JOIN daily_weather
ON STRFTIME('%Y', hired_data.pickup_datetime) = STRFTIME('%Y', daily_weather.day) 
AND STRFTIME('%j', hired_data.pickup_datetime) = STRFTIME('%j', daily_weather.day) 
GROUP BY STRFTIME('%j', hired_data.pickup_datetime) 
ORDER BY daily_weather.HourlyWindSpeed  DESC
LIMIT 10
"""

In [ ]:
res=engine.execute(QUERY_5).fetchall()
# write_query_to_file(QUERY_N, "query_5.sql")
res

### Query 6
Hurricane Sandy in NYC

In [ ]:
QUERY_6 = """
SELECT COUNT(*),hired_data.pickup_datetime, HourlyPrecipitation, HourlyWindSpeed  
FROM 
(
    SELECT tpep_pickup_datetime as pickup_datetime
    FROM taxi_trips WHERE STRFTIME('%Y', tpep_pickup_datetime) = '2012' AND STRFTIME('%m', pickup_datetime) = '10'
                            AND STRFTIME('%d', tpep_pickup_datetime) <= '30'  AND STRFTIME('%d', tpep_pickup_datetime) >= '22'
    UNION ALL
    SELECT pickup_datetime
    FROM uber_trips WHERE STRFTIME('%Y', pickup_datetime) = '2012' AND STRFTIME('%m', pickup_datetime) = '10'
        AND STRFTIME('%d', pickup_datetime) <= '30'  AND STRFTIME('%d', pickup_datetime) >= '22'
) as hired_data
INNER JOIN hourly_weather
ON STRFTIME('%d', hired_data.pickup_datetime) = STRFTIME('%d', hourly_weather.day) 
AND STRFTIME('%H', hired_data.pickup_datetime) = hourly_weather.hour
GROUP BY STRFTIME('%d', hired_data.pickup_datetime), STRFTIME('%H', hired_data.pickup_datetime);
"""

In [ ]:
res=engine.execute(QUERY_6).fetchall()
# write_query_to_file(QUERY_N, "query_6.sql")
res

## Part 4: Visualizing the Data

### Visualization N

_**TODO:** Write some prose that tells the reader what you're about to do here._

_Repeat for each visualization._

_The example below makes use of the `matplotlib` library. There are other libraries, including `pandas` built-in plotting library, kepler for geospatial data representation, `seaborn`, and others._

In [ ]:
def get_data_for_visual_1(engine):
    df = pd.DataFrame(engine.execute(QUERY_1).fetchall())
    
    return df

In [ ]:
def plot_visual_1(dataframe):
    figure, axes = plt.subplots(figsize=(20, 10))
    values = [item[1] for item in pickup_per_hour.values.tolist()]  # use the dataframe to pull out values needed to plot
    x = np.arange(len(list(values)))

    plt.bar(x, values,color=(76/255,114/255,176/255),edgecolor="white",alpha=1, label='num. of trips',zorder=10)
#     plt.yscale("log")
    plt.ylabel("Number of yellow taxi trips taken", labelpad=10.0)
    plt.xlabel("hour in a day", labelpad=10.0)
    plt.xticks(np.arange(24))
#     plt.ylim((0, 1600))

    axes.set_title("Visualization of Query 1 in Part 3")
    
    plt.show()

In [ ]:
pickup_per_hour = get_data_for_visual_1(engine)
plot_visual_1(pickup_per_hour)

In [ ]:
def get_data_for_visual_2(engine):
    
    QUERY_VISUAL_2 = """    
    SELECT hired_data.month, hired_data.distance
    FROM (
    SELECT STRFTIME("%m",pickup_datetime) as month, distance FROM uber_trips
    UNION ALL
    SELECT STRFTIME("%m",tpep_pickup_datetime) as month, trip_distance as distance FROM taxi_trips
    ) as hired_data

    """
    df = pd.DataFrame(engine.execute(QUERY_VISUAL_2).fetchall())
    return df

In [ ]:
# standard_deviation?

In [ ]:
pickup_distance_month = get_data_for_visual_2(engine)

In [ ]:
pickup_distance_month.info()

In [ ]:
pickup_distance_month

In [ ]:
monthly_distance = {}
for i in range(0,12):
    monthly_distance[i]= []

In [ ]:
for row in pickup_distance_month.values.tolist():
    monthly_distance[int(row[0])-1].append(row[1])

In [ ]:
# 独立计算每个月份的confidence？

In [ ]:
means, conf_ls, conf_rs = [], [], []
for key, data in monthly_distance.items():
    import scipy.stats as st
    m = np.mean(data)
    #create 90% confidence interval for population mean weight
    l, r = st.norm.interval(alpha=0.90, loc=np.mean(data), scale=st.sem(data))
    means.append(m)
    conf_ls.append(l)
    conf_rs.append(r)

In [ ]:
# use a more descriptive name for your function
def plot_visual_2(means, conf_ls, conf_rs):
    figure, axes = plt.subplots(figsize=(20, 10))
    
    values = means  # use the dataframe to pull out values needed to plot
    
    # you may want to use matplotlib to plot your visualizations;
    # there are also many other plot types (other 
    # than axes.plot) you can use
    x=np.arange(12)
    axes.plot(x,values)
    axes.fill_between(x, conf_ls, conf_rs, color='b', alpha=.1)
    # there are other methods to use to label your axes, to style 
    # and set up axes labels, etc
    plt.ylabel("Distance: km", labelpad=10.0)
    plt.xticks(np.arange(12),['January','February','March','April','May','June','July','August','September','October','November','December'])
    axes.set_title("average distance traveled per month")
    
    plt.show()

In [ ]:
plot_visual_2(means, conf_ls, conf_rs)

In [ ]:
def get_data_for_visual_5(engine):
    # uber trip中没有 tip amount
    QUERY_VISUAL_2 = """    
    SELECT trip_distance as distance, tip_amount 
    FROM taxi_trips

    """
    df = pd.DataFrame(engine.execute(QUERY_VISUAL_2).fetchall())
    return df

In [ ]:
def plot_visual_5(dataframe):
    figure, axes = plt.subplots(figsize=(20, 10))
    
    distance = [row[0] for row in dataframe.values.tolist()]
    tip = [row[1] for row in dataframe.values.tolist()]
    plt.scatter(x=distance,y=tip, s=5)
    plt.xlabel("Trip Distance: km", labelpad=10.0)
    plt.ylabel("Amount of tips: $", labelpad=10.0)
    axes.set_title("Relationship between tip and distance")
    
    plt.show()

In [ ]:
tip_distance = get_data_for_visual_5(engine)

In [ ]:
plot_visual_5(tip_distance)

In [ ]:
def get_data_for_visual_6(engine):
    # uber trip中没有 tip amount
    QUERY_VISUAL_6 = """    
    SELECT hired_data.tip_amount, HourlyPrecipitation
    FROM (
        SELECT tpep_pickup_datetime as pickup_datetime, tip_amount
        FROM taxi_trips
    ) as hired_data

    INNER JOIN hourly_weather
    ON STRFTIME('%j', hired_data.pickup_datetime) = STRFTIME('%j', hourly_weather.day) 
    AND STRFTIME('%H', hired_data.pickup_datetime) = hourly_weather.hour
    ;

    """
    df = pd.DataFrame(engine.execute(QUERY_VISUAL_6).fetchall())
    return df

In [ ]:
tip_weather = get_data_for_visual_6(engine)
tip_weather

In [ ]:
def plot_visual_6(dataframe):
    figure, axes = plt.subplots(figsize=(20, 10))
    
    tip = [row[0] for row in dataframe.values.tolist()]
    precipitation = [row[1] for row in dataframe.values.tolist()]
    plt.scatter(x=precipitation,y=tip, s=5)
    plt.xlabel("Hourly Precipitation", labelpad=10.0)
    plt.ylabel("Amount of tips: $", labelpad=10.0)
    axes.set_title("Relationship between tip and distance")
    
    plt.show()

In [ ]:
plot_visual_6(tip_weather)